We import what will serve us

In [1]:
from pandas import DataFrame, read_csv
import bigml.api
from bigml.api import BigML
import kaggle
from import_export_json import *

In [2]:
project = 'gmsc'
project_files = 'gmsc_files'
dossier_predictions = './predictions/'
dossier_prep_file = './prep_datasets/'
version = 'v1.3'

In [3]:
project_files = json_load_project(project, version)
project_files

{'prep_data': [{'function': 'remplace_fillna',
   'arguments': {'column': 'MonthlyIncome', 'value': 0}},
  {'function': 'monthly_debt',
   'arguments': {'column': 'MonthlyIncome', 'value': 0}},
  {'function': 'disposable_income', 'arguments': {}},
  {'function': 'process_fillna',
   'arguments': {'column': 'NumberOfDependents', 'value': 0}},
  {'function': 'balanced_income_per_dependent', 'arguments': {}},
  {'function': 'balanced_income', 'arguments': {}},
  {'function': 'weighted_of_late_payment',
   'arguments': {'column': 'NumberOfDependents', 'value': 0}},
  {'function': 'debt_ratio_outlier', 'arguments': {'debt_value': 0.3}}],
 'files': {'kaggle_files': {'train': {'file': 'cs-training.csv',
    'dossier': './'},
   'test': {'file': 'cs-test.csv', 'dossier': './'}},
  'prep_files': {'fulltrain': {'dossier': './prep_datasets/',
    'file_name': 'gmsc_fulltrain_v1.3.csv.bz2'},
   'train': {'dossier': './prep_datasets/',
    'file_name': 'gmsc_train_v1.3.csv.bz2'},
   'dev': {'dossie

In [4]:
csv_suffix = version + '.csv'

filename_fulltrain = 'conf3_fulltrain_gmsc_' + csv_suffix
filename_test = 'conf3_test_gmsc_' + csv_suffix
filename_prediction = 'kaggle_prediction_gmsc_' + csv_suffix

In [5]:
fulltrain = read_csv(project_files['files']['prep_files']['fulltrain']['dossier'] +project_files['files']['prep_files']['fulltrain']['file_name'], index_col=0)
test = read_csv(project_files['files']['prep_files']['test']['dossier'] +project_files['files']['prep_files']['test']['file_name'], index_col=0)

In [6]:
fulltrain = fulltrain[project_files['files']['test_fields_files']['fields_test3']['fields']]
test = test[project_files['files']['test_fields_files']['fields_test3']['fields']]

In [7]:
export_csv = fulltrain.to_csv (dossier_prep_file+ filename_fulltrain,header=True,index_label='Id')
export_csv = test.to_csv (dossier_prep_file+ filename_test,header=True,index_label='Id')

In [8]:
project_files['files']['prep_files']['fulltrain_conf3'] = {'dossier': dossier_prep_file, 'file_name': filename_fulltrain} 
project_files['files']['prep_files']['test_conf3'] = {'dossier': dossier_prep_file, 'file_name': filename_test}
 

## Prediction in validation(test kaggle) set

We keep the best number : 3 

we train the ensemble in all the training test

In [9]:
api = BigML(project=project_files['bigML_ressource']['project'])

In [14]:
source_full = api.create_source(project_files['files']['prep_files']['fulltrain_conf3']['dossier']+project_files['files']['prep_files']['fulltrain_conf3']['file_name'])
# waiting for the source to be finished. Results will be stored in `source`
api.ok(source_full)
print('Source created')
project_files['bigML_ressource']['sources']['fulltrain'] = source_full['resource']

# step 3: creating a dataset from the previously created `source`
dataset_full = api.create_dataset(source_full)
# waiting for the dataset to be finished
api.ok(dataset_full) 
print('Dataset created')
project_files['bigML_ressource']['datasets']['fulltrain'] = dataset_full['resource']

# step 5: creating an Ensemble
ensemble_full = api.create_ensemble(dataset_full)
# waiting for the ensemble to be finished
api.ok(ensemble_full)
print('Model ensemble created')
project_files['bigML_ressource']['models']['ensembles']['fulltrain'] = ensemble_full['resource']
    

Source created
Dataset created
Model ensemble created


Make prediction in the validation (test kaggle) set 

In [17]:
# step 6: creating a source from the data
test_source_validation = api.create_source(project_files['files']['prep_files']['test']['dossier'] +project_files['files']['prep_files']['test']['file_name'])
# waiting for the source to be finished. Results will be stored in `source`
api.ok(test_source_validation)
print('Source created')
project_files['bigML_ressource']['sources']['test'] = test_source_validation['resource']
# step 8: creating a dataset from the previously created `source`
test_dataset_validation = api.create_dataset(test_source_validation)
# waiting for the dataset to be finished
api.ok(test_dataset_validation)
print('Dataset created')
project_files['bigML_ressource']['datasets']['test'] = test_dataset_validation['resource']
# step 10: creating a batch prediction
batch_prediction_validation = api.create_batch_prediction(ensemble_full, test_dataset_validation,
                                               {"all_fields": True,
                                                "header": True,
                                                "confidence": True,
                                                "probabilities": True})
# waiting for the batch_prediction to be finished
api.ok(batch_prediction_validation)
print('Batch prediction created')
# downloading the results to your computer
api.download_batch_prediction(batch_prediction_validation, filename=dossier_predictions+'ensemble_'+filename_prediction)
project_files['files']['predictions']['ensembles']['kaggle_test_prediction'] = {'dossier': dossier_predictions, 'file_name': 'ensemble_'+filename_prediction}
    

Source created
Dataset created
Batch prediction created


We recover the prediction

In [18]:
prediction=read_csv(dossier_predictions+'ensemble_'+filename_prediction,index_col=False)
print(list(prediction))

['Id', 'SeriousDlqin2yrs', 'RevolvingUtilizationOfUnsecuredLines', 'age', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome', 'MonthlyDebt', 'DisposableIncome', 'BalancedIncome', 'BalancedIncomePerDependent', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'WeightedOfLatePayment', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents', 'SeriousDlqin2yrs.1', 'confidence', '0 probability', '1 probability']


We rename the columns for rendering

In [19]:
kaggle_prediction=DataFrame()
kaggle_prediction['Id']=prediction['Id']
kaggle_prediction['Probability']=prediction['1 probability']

We check the output

In [20]:
kaggle_prediction.head()

,Id,Probability
0,1,0.04705
1,2,0.04610
2,3,0.01456
3,4,0.08689
4,5,0.11611


Render preparation and sending for evaluation by Kaggle

In [22]:
kaggle_prediction_file=dossier_predictions+'ensemble_'+filename_prediction
kaggle_prediction.to_csv(kaggle_prediction_file,index=False)
kaggle.api.competition_submit(kaggle_prediction_file, "Prediction Bigml v1.3", "GiveMeSomeCredit")

100%|██████████| 1.67M/1.67M [00:06<00:00, 252kB/s] 


Successfully submitted to Give Me Some Credit

Maud SONET